In [39]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame
import os
import joblib
import sklearn
from sklearn.svm import SVR
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Import plotting libraries
import seaborn as sns
import matplotlib 
from matplotlib import pyplot as plt

# Set larger fontsize for all plots
matplotlib.rcParams.update({'font.size': 30})
from IPython.display import clear_output

In [40]:
# Getting dataset
df = pd.read_csv('../raw data.csv')
df

,DOI,method,metal,metal_source,metal_amount_mmol,ligand,ligand_amount_mmol,reductant,reductant_amount_mmol,solvent1,sol1_vol_mL,solvent2,solv2_vol_mL,time_min,temp_c,size_nm,Absorbance,Ellip_mdeg,g_factor*10^4,g_factor*10^4.1
0,10.1021/cm3033725,one-pot reduction,gold,Tetrachloroauric(III) acid,0.132,N-acetyl-L-cysteine,0.53,sodium borohydride,1.32,water,32.5,None,0.0,720,0,3.06,1.1,0,None,0
1,None,one-pot reduction,gold,Tetrachloroauric(III) acid,0.132,N-acetyl-L-cysteine,0.53,sodium borohydride,1.32,water,33.5,None,0.0,720,0,1,1,2.5,None,0.757989206
2,None,one-pot reduction,silver,silver nitrate,0.870,glutathione,0.97,sodium borohydride,9.99,water,30.0,None,0.0,60,0,1,1.3,2.5,None,0.58306862
3,None,one-pot reduction,silver,silver nitrate,0.930,N-acetyl-Lcysteine,1.00,sodium borohydride,10.20,water,37.0,None,0.0,60,0,2.37,2,0,None,0
4,10.1002/adfm.201001120,one-pot reduction,gold,Tetrachloroauric(III) acid,0.132,glutathione,0.53,sodium borohydride,1.32,water,32.5,None,0.0,720,0,4,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,None,one-pot reduction,silver,silver nitrate,0.200,Dihydrolipoic acid,0.10,sodium borohydride,0.50,water,9.0,None,0.0,480,25,None,None,None,2,2
198,10.1021/la703351p,one-pot reduction,silver,silver nitrate,0.500,penicillamine,0.50,sodium borohydride,5.00,water,30.0,methanol,25.0,90,25,1.05,None,None,15,15
199,None,one-pot reduction,silver,silver nitrate,0.500,penicillamine,0.50,sodium borohydride,5.00,water,30.0,methanol,25.0,90,25,1.3,None,None,10,10
200,None,one-pot reduction,silver,silver nitrate,0.500,penicillamine,0.25,sodium borohydride,5.00,water,30.0,methanol,25.0,90,25,1.05,None,None,7,7


In [41]:
#Checks if there are any columns with empty cells
df.isna().sum()

DOI                      0
method                   0
metal                    0
metal_source             0
metal_amount_mmol        0
ligand                   0
ligand_amount_mmol       0
reductant                0
reductant_amount_mmol    0
solvent1                 0
sol1_vol_mL              0
solvent2                 0
solv2_vol_mL             0
time_min                 0
temp_c                   0
size_nm                  0
Absorbance               0
Ellip_mdeg               0
g_factor*10^4            0
g_factor*10^4.1          0
dtype: int64

# Scaling and Transforming

In [42]:
df_input = df.drop(columns =['DOI','method','metal_source','size_nm','Absorbance','Ellip_mdeg','g_factor*10^4.1','g_factor*10^4'], inplace = False, axis = 1)
df_output = df[['size_nm','g_factor*10^4.1']]

In [43]:
#Checks the column names, and ensures that they do not have any leading or trailing spaces
df_input.columns = df_input.columns.str.strip()
df_output.columns = df_output.columns.str.strip()

In [44]:
df_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   metal                  202 non-null    object 
 1   metal_amount_mmol      202 non-null    float64
 2   ligand                 202 non-null    object 
 3   ligand_amount_mmol     202 non-null    float64
 4   reductant              202 non-null    object 
 5   reductant_amount_mmol  202 non-null    float64
 6   solvent1               202 non-null    object 
 7   sol1_vol_mL            202 non-null    float64
 8   solvent2               202 non-null    object 
 9   solv2_vol_mL           202 non-null    float64
 10  time_min               202 non-null    int64  
 11  temp_c                 202 non-null    int64  
dtypes: float64(5), int64(2), object(5)
memory usage: 19.1+ KB


In [45]:
#change temp and time from integer to float
df_input['temp_c'] = df_input['temp_c'].astype(float)
df_input['time_min'] = df_input['time_min'].astype(float)
df_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   metal                  202 non-null    object 
 1   metal_amount_mmol      202 non-null    float64
 2   ligand                 202 non-null    object 
 3   ligand_amount_mmol     202 non-null    float64
 4   reductant              202 non-null    object 
 5   reductant_amount_mmol  202 non-null    float64
 6   solvent1               202 non-null    object 
 7   sol1_vol_mL            202 non-null    float64
 8   solvent2               202 non-null    object 
 9   solv2_vol_mL           202 non-null    float64
 10  time_min               202 non-null    float64
 11  temp_c                 202 non-null    float64
dtypes: float64(7), object(5)
memory usage: 19.1+ KB


In [46]:
#Initializes 2 lists to contain all the numerical and categorical input columns
input_num_cols = [col for col in df_input.columns if df[col].dtypes !='O']
input_cat_cols = [col for col in df_input.columns if df[col].dtypes =='O']
print(input_num_cols, input_cat_cols)

['metal_amount_mmol', 'ligand_amount_mmol', 'reductant_amount_mmol', 'sol1_vol_mL', 'solv2_vol_mL', 'time_min', 'temp_c'] ['metal', 'ligand', 'reductant', 'solvent1', 'solvent2']


In [47]:
#Initializes the ColumnTransformer object, and specifies what it will do with a dataframe
#scaling numerical columns
#onehotencoder creates a binary column for each categorical entry
ct = ColumnTransformer([
    ('step1', StandardScaler(), input_num_cols),
    ('step2', OneHotEncoder(sparse=False, handle_unknown='ignore'), input_cat_cols)
], remainder = 'passthrough')
#what are the transformers in ct
ct.transformers

[('step1',
  StandardScaler(),
  ['metal_amount_mmol',
   'ligand_amount_mmol',
   'reductant_amount_mmol',
   'sol1_vol_mL',
   'solv2_vol_mL',
   'time_min',
   'temp_c']),
 ('step2',
  OneHotEncoder(handle_unknown='ignore', sparse=False),
  ['metal', 'ligand', 'reductant', 'solvent1', 'solvent2'])]

In [48]:
#Uses the ColumnTransformer object to modify the input columns
df_input_scaled_encoded = pd.DataFrame(ct.fit_transform(df_input))
df_input_scaled_encoded

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
0,-0.454677,-0.189866,-0.196766,-0.120691,-0.523499,0.681196,-0.998532,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,-0.454677,-0.189866,-0.196766,-0.100205,-0.523499,0.681196,-0.998532,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.606371,0.030694,0.275068,-0.171905,-0.523499,-0.595993,-0.998532,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.773936,0.045732,0.286496,-0.028504,-0.523499,-0.595993,-0.998532,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,-0.454677,-0.189866,-0.196766,-0.120691,-0.523499,0.681196,-0.998532,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,-0.264770,-0.405413,-0.241391,-0.602108,-0.523499,0.216764,-0.175924,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
198,0.573054,-0.204904,0.003505,-0.171905,-0.003814,-0.537939,-0.175924,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
199,0.573054,-0.204904,0.003505,-0.171905,-0.003814,-0.537939,-0.175924,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.573054,-0.330222,0.003505,-0.171905,-0.003814,-0.537939,-0.175924,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [49]:
#Number of numerical columns
len(input_num_cols)

7

In [50]:
#renaming columns
for i in range(len(input_num_cols)):
    df_input_scaled_encoded.rename(columns={df_input_scaled_encoded.columns[i]: input_num_cols[i]}, inplace = True)

In [51]:
df_input_scaled_encoded.columns

Index([    'metal_amount_mmol',    'ligand_amount_mmol',
       'reductant_amount_mmol',           'sol1_vol_mL',
                'solv2_vol_mL',              'time_min',
                      'temp_c',                       7,
                             8,                       9,
       ...
                           119,                     120,
                           121,                     122,
                           123,                     124,
                           125,                     126,
                           127,                     128],
      dtype='object', length=129)

In [52]:
#appends output columns
df_scaled_encoded = pd.concat([df_input_scaled_encoded, df_output], axis = 1)
df_scaled_encoded

,metal_amount_mmol,ligand_amount_mmol,reductant_amount_mmol,sol1_vol_mL,solv2_vol_mL,time_min,temp_c,7,8,9,...,121,122,123,124,125,126,127,128,size_nm,g_factor*10^4.1
0,-0.454677,-0.189866,-0.196766,-0.120691,-0.523499,0.681196,-0.998532,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,3.06,0
1,-0.454677,-0.189866,-0.196766,-0.100205,-0.523499,0.681196,-0.998532,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1,0.757989206
2,1.606371,0.030694,0.275068,-0.171905,-0.523499,-0.595993,-0.998532,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1,0.58306862
3,1.773936,0.045732,0.286496,-0.028504,-0.523499,-0.595993,-0.998532,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.37,0
4,-0.454677,-0.189866,-0.196766,-0.120691,-0.523499,0.681196,-0.998532,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,4,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,-0.264770,-0.405413,-0.241391,-0.602108,-0.523499,0.216764,-0.175924,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,None,2
198,0.573054,-0.204904,0.003505,-0.171905,-0.003814,-0.537939,-0.175924,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.05,15
199,0.573054,-0.204904,0.003505,-0.171905,-0.003814,-0.537939,-0.175924,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.3,10
200,0.573054,-0.330222,0.003505,-0.171905,-0.003814,-0.537939,-0.175924,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.05,7


In [53]:
df_scaled_encoded.to_csv('dataset_scaled.csv')

# Making separate outputs dataset

In [54]:
#Saves into a list the row indexes to drop for size dataset
total_row_num = len(df_scaled_encoded)
drop_list_size =[]
for row_i in range(total_row_num):
    if df_scaled_encoded['size_nm'].values[row_i] == 'None':
        drop_list_size.append(row_i)
    
#number of entries
print(total_row_num-len(drop_list_size))

144


In [55]:
#Drops rows that don't have size output
df_size_scaled_encoded = df_scaled_encoded.drop(drop_list_size)
df_size_scaled_encoded.to_csv('dataset_scaled_size.csv')